In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
from datetime import datetime
import time
from selenium.webdriver.common.by import By


from openpyxl.styles import PatternFill
from openpyxl import Workbook
from random import uniform

import os
current_directory = os.getcwd()

In [4]:
#크롤링 대상 계정
#교육부 학교폭력 / 성폭력 상담
#한국청소년상담복지개발원
#청소년폭력예방재단
#서울시청소년지원센터 

In [17]:
# ex)한국청소년상담복지개발원
# 코드 참고 : https://github.com/catSirup/naver_kin_crawling
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=chrome_options)

keyword = '사이버 폭력'

idx_num = 1
page_index = 1
date = datetime.now().strftime("%Y-%m-%d_%H_%M_%S")


url_list_filename = current_directory + f"/result/url_list_{keyword.replace(' ', '+')}_한국청소년상담복지개발원.txt"
with open(url_list_filename, 'w') as f:
    page_url = []

    while True:
        time.sleep(uniform(0.01, 1.0))

        url = f'https://kin.naver.com/userinfo/answerList.naver?u=6oCfeacJKVQGwsBB7OHEIS4miiPixwQ%2FoueervNeYrg%3D&isSearch=true&query=%EC%82%AC%EC%9D%B4%EB%B2%84%20%ED%8F%AD%EB%A0%A5&sd=answer&y=0&section=qna&isWorry=false&x=0&page={page_index}'
        driver.get(url)
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        tags = soup.find_all('td', {'class': 'title'})

        for tag in tags:
            url = tag.a.get('href').replace('amp;', '')
            page_url.append(url)
            f.write(url + "\n")

        if idx_num != page_index : 
            page_index += 1
        else:
            break
            
filename = f'result/{keyword.replace(" ", ".")}_한국청소년상담복지개발원_crawling_result.xlsx'

wb = Workbook()
sheet = wb.active
sheet.append(['제목', '질문', '답변'])

for j in range(1, 4):
    sheet.cell(row=1, column=j).fill = PatternFill(start_color='808080', end_color='808080', fill_type='solid')

for i in page_url:
    driver.get(i)
    title = driver.find_element(By.CLASS_NAME, 'title').text

    try:
        question_txt = driver.find_element(By.CLASS_NAME, 'c-heading__content').text
    except:
        question_txt = ""

    answer_list = driver.find_elements(By.CLASS_NAME, "answer-content__list._answerList")

    for n, answer in enumerate(answer_list):
        context = answer.find_elements(By.CLASS_NAME, "_endContentsText.c-heading-answer__content-user")
        for context_body in context:
            if '청소년사이버상담센터' in context_body.text:
                sheet.append([title, question_txt, context_body.text])

    wb.save(filename)

driver.close()  